# Наивная модель
### векторизация слов на основе TfIdf и классификатор RandomForest

Классификация текста — классическая задача в области обработки естественного языка. В качестве наивной модел будем использовать подход на основе TF-IDF. 
До победы ембедингов данный подход был основным.
При таком подходе на качество классификации больше  влияют методы предобработки и особенности текста. Будем считать что в рамках diplom_EDA я сделал все необходимое в этом направлении.  

На входе у нас список из обращений в техническую поддержку и список размеченных классов. Далее делаем Pipeline, который включает в себя векторизацию слов на основе TfIdf и классификатор RandomForest.

## SETUP

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# загружаем библиотеки
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает 
# написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность 
# использования каждого слова из некоторого набора слов (количество слов набора определяет 
# размерность вектора) в каждом тексте

from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров




In [67]:
PATH = 'drive/MyDrive/data/diplom_final/'
# PATH = ''

## EDA ...

## Load preprocessing data

In [68]:
# загружаем предобработанные данные
df = pd.read_csv(PATH+'data_prepare.csv', sep=',')
df.head()

,class,description,numbers
0,Поддержка товаров,товар коллега клиент позвонить сообщить товар ...,25
1,Fiji,проблема открытие зацепка открытие зацепка поя...,16
2,Export buildman,отправляться локальный проверка отправка локал...,12
3,API,находиться адрес число название представлять к...,37
4,Личный Кабинет,заходить лк попасть личный кабинет делать этаж...,8


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9811 entries, 0 to 9810
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   class        9811 non-null   object
 1   description  9811 non-null   object
 2   numbers      9811 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 230.1+ KB


In [70]:
df['class'].nunique()

12

In [71]:
df.head()

,class,description,numbers
0,Поддержка товаров,товар коллега клиент позвонить сообщить товар ...,25
1,Fiji,проблема открытие зацепка открытие зацепка поя...,16
2,Export buildman,отправляться локальный проверка отправка локал...,12
3,API,находиться адрес число название представлять к...,37
4,Личный Кабинет,заходить лк попасть личный кабинет делать этаж...,8


In [72]:
y = df['class']
X = df['description']

Разделим датасет на тренировочнуюи и тестовую части:

In [73]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=True, random_state=42)

## Model

In [74]:
model_naive = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier())
                     ])

In [75]:
model_naive.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

## Evaluate

In [76]:
model_naive.predict(['прошу удалить отзыв о нашей компнии так он не соответвует действительности'])[0]

'Поддержка по отзывам'

In [77]:
predicted_rf = model_naive.predict(X_test)
print(metrics.classification_report(predicted_rf, y_test))

                          precision    recall  f1-score   support

                     API       0.92      0.93      0.93        87
               Atlassian       0.89      0.93      0.91        75
               CRM отдел       0.90      0.85      0.87       101
         Export buildman       0.90      0.81      0.86        81
                    Fiji       0.86      0.78      0.82        88
                      IR       0.85      0.91      0.88        76
          Группа Digital       0.78      0.87      0.82        71
    Группа корректуры РМ       0.83      0.92      0.87        78
          Личный Кабинет       0.95      0.87      0.91        84
    Поддержка по отзывам       0.97      0.95      0.96       104
       Поддержка товаров       0.93      0.93      0.93        74
Телекомунникация и связь       0.89      0.92      0.91        63

                accuracy                           0.89       982
               macro avg       0.89      0.89      0.89       982
        

In [78]:
print('Model Accuracy - train data:', model_naive.score(X_train, y_train))
print('Model Accuracy - test data:', model_naive.score(X_test, y_test))

Model Accuracy - train data: 1.0
Model Accuracy - test data: 0.890020366598778


Результат "наивной" модели - 0.88.  
Возьмем это за основу.

Так как это "наивная" модель, то более тонкую настройку выполнять не будем.  
Но в качестве потенциальных направления можно рассматривать:  
* дальнейшая предобработка текста 
* подбор параметров TfIdf:
* попробовать другой классификатор (SGD, SVM , XGB итд.)

### Подход 2. Развитие наивной модели

Создадим дополнительно классификатора (чтобы можно было сравнить качество получившихся моделей) и обучим их на тестовых данных:

In [79]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('knb_clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=N

Предскажем получившимися моделями класс текстов в тестовой выборке и сравним метрики:

In [80]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

                          precision    recall  f1-score   support

                     API       0.95      0.91      0.93        92
               Atlassian       0.89      0.93      0.91        75
               CRM отдел       0.90      0.86      0.88       100
         Export buildman       0.92      0.86      0.89        78
                    Fiji       0.96      0.88      0.92        88
                      IR       0.85      0.90      0.87        77
          Группа Digital       0.76      0.91      0.83        67
    Группа корректуры РМ       0.90      0.87      0.88        90
          Личный Кабинет       0.92      0.90      0.91        79
    Поддержка по отзывам       0.99      0.96      0.98       105
       Поддержка товаров       0.89      0.96      0.92        69
Телекомунникация и связь       0.89      0.94      0.91        62

                accuracy                           0.90       982
               macro avg       0.90      0.91      0.90       982
        

In [81]:
predicted_knb = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_knb, y_test))

                          precision    recall  f1-score   support

                     API       0.88      0.83      0.85        93
               Atlassian       0.80      0.84      0.82        75
               CRM отдел       0.86      0.78      0.82       107
         Export buildman       0.84      0.80      0.82        76
                    Fiji       0.85      0.80      0.82        85
                      IR       0.70      0.83      0.76        69
          Группа Digital       0.68      0.82      0.74        66
    Группа корректуры РМ       0.92      0.80      0.86       100
          Личный Кабинет       0.79      0.73      0.76        84
    Поддержка по отзывам       0.96      0.97      0.97       101
       Поддержка товаров       0.81      0.94      0.87        64
Телекомунникация и связь       0.80      0.84      0.82        62

                accuracy                           0.83       982
               macro avg       0.82      0.83      0.82       982
        

Из двух дополнительно подключенных классификаторов выбирам вариант на основе SGD (Stochastic gradient descent).  
Попробуем улучшить нашу модель, используя различные параметры

In [ ]:
parameters = { 
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              'tfidf__strip_accents':['ascii', 'unicode', None],
               'tfidf__ngram_range':[(1,2), (2,3), (3,4)]
              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train, y_train)
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.9044057774012858 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': 'l2', 'tfidf__ngram_range': (1, 2), 'tfidf__strip_accents': 'unicode'}


Обучим модель, используя предложенные параметры и оценим ее качество на тестовых данных:



In [84]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), strip_accents='unicode')),
    ('sgd_clf', SGDClassifier(penalty='elasticnet', class_weight='balanced', random_state=42))])
sgd_ppl_clf.fit(X_train, y_train)
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

                          precision    recall  f1-score   support

                     API       0.95      0.93      0.94        90
               Atlassian       0.90      0.91      0.90        78
               CRM отдел       0.89      0.84      0.86       101
         Export buildman       0.90      0.88      0.89        75
                    Fiji       0.95      0.89      0.92        85
                      IR       0.85      0.92      0.88        75
          Группа Digital       0.76      0.94      0.84        65
    Группа корректуры РМ       0.90      0.88      0.89        89
          Личный Кабинет       0.94      0.90      0.92        80
    Поддержка по отзывам       0.99      0.96      0.98       105
       Поддержка товаров       0.92      0.93      0.93        73
Телекомунникация и связь       0.91      0.89      0.90        66

                accuracy                           0.91       982
               macro avg       0.90      0.91      0.90       982
        

In [85]:
print('Model Accuracy - train data:', sgd_ppl_clf.score(X_train, y_train))
print('Model Accuracy - test data:', sgd_ppl_clf.score(X_test, y_test))

Model Accuracy - train data: 0.9745158002038736
Model Accuracy - test data: 0.9063136456211812


Итог: лучшее значение Accurency - 0.9063